In [1]:
import vaex as vx

In [2]:
DATA = './data/nyc-taxi/*.parquet'

In [3]:
%%time

df = vx.open(DATA)

CPU times: user 256 ms, sys: 65.1 ms, total: 321 ms
Wall time: 417 ms


In [4]:
df.column_names

[
    'VendorID',
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'passenger_count',
    'trip_distance',
    'RatecodeID',
    'store_and_fwd_flag',
    'PULocationID',
    'DOLocationID',
    'payment_type',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'airport_fee'
]

In [5]:
df.shape

(176408393, 19)

In [6]:
df.drop('store_and_fwd_flag').byte_size() / 1e9  # trying to size string col raises error

25.402808592

In [6]:
df.head()

#,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,--,--
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,--,--
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,--,--
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,--,--
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,--,--
5,2,2018-11-28 16:25:49,2018-11-28 16:28:26,5,0,1,N,193,193,2,3.5,0.5,0.5,0,5.76,0.3,13.31,--,--
6,2,2018-11-28 16:29:37,2018-11-28 16:33:43,5,0,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,--,--
7,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,N,163,229,1,6.5,0.5,0.5,1.25,0,0.3,9.05,--,--
8,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,N,229,7,1,13.5,0.5,0.5,3.7,0,0.3,18.5,--,--
9,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,N,141,234,1,10,0.5,0.5,1.7,0,0.3,13,--,--


In [7]:
%%time

df['is_congested'] = df.congestion_surcharge.fillna(0) > 0

df.groupby(
    'DOLocationID',
    agg={
        'mean_fare': vx.agg.mean('fare_amount'),
        'count': vx.agg.count('*'),
        'congestion': vx.agg.mean('is_congested')
    },
    progress=True
).sort('count', ascending=False)

groupby [----------------------------------------]  0.00% estimated time: unknown                 

[03/04/23 14:00:00] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/ast.py:4 ]8;id=585335;file:///opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=290884;file:///opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/site-packages/astropy/logger.py#207\207]8;;\
                             07: DeprecationWarning: visit_NameConstant is deprecated; add                         
                             visit_Constant                                                                        
                               return visitor(node)                                                                
                                                                                                                   

groupby [####################--------------------] 50.00% estimated time:     1.45s =  0.0m =  0.0h 

[03/04/23 14:00:02] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/ast.py:4 ]8;id=803444;file:///opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/site-packages/astropy/logger.py\logger.py]8;;\:]8;id=708617;file:///opt/homebrew/Caskroom/miniforge/base/envs/paysim/lib/python3.9/site-packages/astropy/logger.py#207\207]8;;\
                             07: DeprecationWarning: visit_Num is deprecated; add visit_Constant                   
                               return visitor(node)                                                                
                                                                                                                   

groupby [########################################] 100.00% elapsed time  :     5.20s =  0.1m =  0.0h
 CPU times: user 8.78 s, sys: 1.36 s, total: 10.1 s
Wall time: 5.25 s


#,DOLocationID,mean_fare,count,congestion
0,236,9.535313307219667,7540155,0.9300530294138516
1,237,9.245066620242751,7102646,0.9409542584552292
2,161,10.944027083168406,6428273,0.9353692663643874
3,170,10.735887985246539,5376756,0.9321483065253473
4,162,11.09996795439159,4948884,0.934476338503792
...,...,...,...,...
259,2,40.195150214592275,233,0.08583690987124463
260,105,16.492748091603055,131,0.7404580152671756
261,199,12.733333333333333,15,0.26666666666666666
262,110,63.9375,8,0.25


In [8]:
%%time

with vx.progress.tree('rich'):
    df['is_congested'] = df.congestion_surcharge.fillna(0) > 0

    grouped = df.groupby(
        'DOLocationID',
        agg={
            'mean_fare': vx.agg.mean('fare_amount'),
            'count': vx.agg.count('*'),
            'congestion': vx.agg.mean('is_congested')
        },
        progress=True
    ).sort('count', ascending=False)
    
grouped

Output()

groupby [########################################] 100.00% elapsed time  :     5.15s =  0.1m =  0.0h
 

CPU times: user 8.89 s, sys: 1.31 s, total: 10.2 s
Wall time: 5.22 s


#,DOLocationID,mean_fare,count,congestion
0,236,9.535313307219795,7540155,0.9300530294138516
1,237,9.24506662024279,7102646,0.9409542584552292
2,161,10.944027083168478,6428273,0.9353692663643874
3,170,10.735887985246611,5376756,0.9321483065253473
4,162,11.099967954391609,4948884,0.934476338503792
...,...,...,...,...
259,2,40.19515021459228,233,0.08583690987124463
260,105,16.492748091603055,131,0.7404580152671756
261,199,12.733333333333333,15,0.26666666666666666
262,110,63.9375,8,0.25
